In [112]:
%matplotlib inline

from clipper.src import count_features
from gscripts.general import region_helpers
import pandas as pd
from gscripts.general import parsers
from IPython.core.display import HTML

In [2]:
region_counter = count_features.RegionCounter("mm9")

In [104]:
!mkdir -p /nas3/gpratt/projects/fet_family/analysis/mouse_clip_v5
!scp tscc-login.sdsc.edu:/home/gpratt/projects/fet_family/analysis/mouse_clip_v5/*WholeBrain.merged.bam /nas3/gpratt/projects/fet_family/analysis/mouse_clip_v5
!scp -q tscc-login.sdsc.edu:/home/gpratt/projects/fet_family/analysis/mouse_clip_v5/*.{metrics,final.out,peaks.bed} /nas3/gpratt/projects/fet_family/analysis/mouse_clip_v5 

TAF15_WholeBrain.merged.bam                   100% 1208MB  25.2MB/s   00:48    
TDP43_WholeBrain.merged.bam                   100%  427MB  30.5MB/s   00:14    
TLS_WholeBrain.merged.bam                     100%  240MB  26.7MB/s   00:09    


In [113]:
clip_rnaseq_df = parsers.clipseq_metrics("/nas3/gpratt/projects/fet_family/analysis/mouse_clip_v5", iclip=False)
filtered_df = clip_rnaseq_df[[ "Input Reads",
                             "Too short reads",
                             "Reads that were too short percent",
                             'repetitive_count',
                        "Reads Passing Quality Filter", 
                        "Uniquely Mapped Reads",
                        "Uniquely mapped reads %",
                        "Usable Reads",
                            
                             
                            #"spot",
                            "Num Peaks",
                            #"Fraction Collapsed",
                            # "Fraction Usable"                             
                            ]]


HTML(filtered_df.to_html(formatters={"Input Reads" : parsers.commas,
                                     'repetitive_count': parsers.commas,
                                     "Too short reads": parsers.commas,
                                     "Reads Passing Quality Filter" : parsers.commas,
                                     "Reads after Quality Filtering" : parsers.commas,
                                     "Uniquely Mapped Reads" : parsers.commas,
                                     "Usable Reads" : parsers.commas,
                                     "Num Peaks": parsers.commas
                                     } ))

,Input Reads,Too short reads,Reads that were too short percent,repetitive_count,Reads Passing Quality Filter,Uniquely Mapped Reads,Uniquely mapped reads %,Usable Reads,Num Peaks
20090209_MP_none1_mouse_brain_CLIPseq_TDP43_lower_band,"4,472,151","4,062",0.1,"1,267,649","3,993,240","1,354,679",33.92%,"216,923","2,246"
20090209_MP_none2_mouse_brain_3A_CLIPseq_TLS,"5,940,464",661,0.0,"655,254","5,792,395","1,866,037",32.22%,"1,694,735","4,840"
20090601_MP_none3_mouse_brain_2A_CLIPseq_TLS,"6,754,559","10,796",0.2,"342,212","6,661,709","1,223,067",18.36%,"1,096,610","3,181"
20090601_MP_none4_mouse_brain_10A_CLIPseq_TDP43_highMNase,"5,341,577",763,0.0,"806,946","5,185,955","735,875",14.19%,"520,223","20,426"
20090601_MP_none5_mouse_brain_13E_CLIPseq_empty_beads,"6,342,111",961,0.0,"1,095,800","5,963,457","1,086,740",18.22%,"132,641","5,717"
20091230_MP10_1_TLS_clip_mouse_brain_3A,"8,321,534","1,094",0.0,"46,569","8,308,383","282,219",3.40%,"266,699",535
20091230_MP11_1_TLS_clip_mouse_brain_3B,"11,679,461","1,907",0.0,"194,283","11,627,895","1,076,942",9.26%,"996,634","3,153"
20101110_MP41_1_mouse_brain_CLIPseq_TDP43_lower_band,"70,496,200","684,826",1.0,"20,292,044","65,591,046","18,875,696",28.78%,"3,412,759","71,138"
20101110_MP42_1_mouse_brain_CLIPseq_TDP43_upper_band,"71,776,915","674,998",0.9,"29,352,592","65,012,667","19,378,128",29.81%,"4,407,855","58,207"
TAF15_1,"288,984,903","11,548,509",4.0,"50,320,764","262,881,597","122,174,227",46.48%,"22,801,935","33,459"


In [114]:
def gene_id_to_type_fun(genes):
    for gene_id in genes:
        try:
            yield gene_id_to_type[gene_id]
        except:
            yield "no_type"

In [115]:
gene_id_to_name = region_helpers.gene_id_to_name("/nas3/gpratt/gencode/gencode.vM1.annotation.gtf.db")
gene_id_to_type = region_helpers.gene_id_to_type("/nas3/gpratt/gencode/gencode.vM1.annotation.gtf.db")

name_to_gene_id = {value: key for key, value in gene_id_to_name.items()}

In [ ]:
taf15_df = region_counter.count_features("/nas3/gpratt/projects/fet_family/analysis/mouse_clip_v5/TAF15_WholeBrain.merged.bam")
tdp43_df = region_counter.count_features("/nas3/gpratt/projects/fet_family/analysis/mouse_clip_v5/TDP43_WholeBrain.merged.bam")
tls_df = region_counter.count_features("/nas3/gpratt/projects/fet_family/analysis/mouse_clip_v5/TLS_WholeBrain.merged.bam")

taf15_df = pd.DataFrame(taf15_df.fillna(0).stack(), columns=["count"])
tdp43_df= pd.DataFrame(tdp43_df.fillna(0).stack(), columns=["count"])
tls_df = pd.DataFrame(tls_df.fillna(0).stack(), columns=["count"])

taf15_df['gene_type'] = list(gene_id_to_type_fun(taf15_df.index.get_level_values(level=1)))
tdp43_df['gene_type'] = list(gene_id_to_type_fun(tdp43_df.index.get_level_values(level=1)))
tls_df['gene_type'] = list(gene_id_to_type_fun(tls_df.index.get_level_values(level=1)))

In [ ]:
non_coding_genes = {'IG_C_gene',
 'IG_D_gene',
 'IG_J_gene',
 'IG_V_gene',
 'Mt_rRNA',
 'Mt_tRNA',
 'lincRNA',
 'miRNA',
 'misc_RNA',
 'polymorphic_pseudogene',
 'processed_transcript',
 'pseudogene',
 'rRNA',
 'snRNA',
 'snoRNA'}

In [ ]:
total

In [ ]:
total = taf15_df['count'].sum()

In [ ]:
total_protein_coding = taf15_df[taf15_df['gene_type'] == "protein_coding"]['count'].sum() 

In [ ]:
total_noncoding = taf15_df[taf15_df['gene_type'].isin(non_coding_genes)]['count'].sum()

In [ ]:
total_protein_coding

In [ ]:
total_protein_coding / total

In [ ]:
total_noncoding

In [ ]:
total_noncoding / total

In [ ]:
taf15_premrna = taf15_df[taf15_df['gene_type'] == "protein_coding"].ix[['proxintron500', 'distintron500']]

In [ ]:
taf15_premrna['count'].sum()

In [ ]:
len(set(taf15_premrna[taf15_premrna['count'] > 10].index.get_level_values(level=1)))

In [ ]:
taf15_df[taf15_df.gene_type == "no_type"]['count'].sum()

In [ ]:
taf15_df[taf15_df.gene_type == "no_type"]['count'].sum() / total